Find out appropriate algorithm for training for 
your dataset. 
Implement selected Algorithm without use of 
Library.  & Test the model on the test dataset and 
compute metrics. 
Check for overfi ng or underfi ng. 
Hyperparameter Tuning 

In [2]:
import pandas as pd
df=pd.read_csv('cardio_train_cleaned.csv')
df.head()

,Unnamed: 0,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,0,50.0,2,168,62.0,110,80,1,1,0,0,1,0
1,1,1,55.0,1,156,85.0,140,90,3,1,0,0,1,1
2,2,2,51.0,1,165,64.0,130,70,3,1,0,0,0,1
3,3,3,48.0,2,169,82.0,150,100,1,1,0,0,1,1
4,4,4,47.0,1,156,56.0,100,60,1,1,0,0,0,0


In [12]:
df.describe()

,Unnamed: 0,id,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio
count,68649.000000,68649.000000,68649.000000,68649.000000,68649.000000,68649.000000,68649.000000,68649.000000,68649.000000,68649.000000,68649.000000,68649.000000
mean,35007.542018,49983.909394,52.829131,0.348701,164.360748,74.120868,126.680607,81.307958,0.087998,0.053373,0.803347,0.494792
std,20202.291718,28843.977870,6.769596,0.476563,8.184709,14.332894,16.729881,9.456264,0.283295,0.224778,0.397470,0.499977
min,0.000000,0.000000,29.000000,0.000000,55.000000,11.000000,60.000000,40.000000,0.000000,0.000000,0.000000,0.000000
25%,17505.000000,25013.000000,48.000000,0.000000,159.000000,65.000000,120.000000,80.000000,0.000000,0.000000,1.000000,0.000000
50%,35017.000000,50024.000000,53.000000,0.000000,165.000000,72.000000,120.000000,80.000000,0.000000,0.000000,1.000000,0.000000
75%,52492.000000,74877.000000,58.000000,1.000000,170.000000,82.000000,140.000000,90.000000,0.000000,0.000000,1.000000,1.000000
max,69999.000000,99999.000000,64.000000,1.000000,250.000000,200.000000,401.000000,182.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
df = df.drop(columns=['Unnamed: 0', 'id'])
X = df.drop(columns=['cardio'])
y = df['cardio']


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from sklearn.preprocessing import StandardScaler
import joblib

scaler = StandardScaler()

num_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']

scaler.fit(X_train[num_cols])

X_train[num_cols] = scaler.transform(X_train[num_cols])
X_test[num_cols]  = scaler.transform(X_test[num_cols])



In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

param_rf = {
    'n_estimators': [100, 200],
    'max_depth': [10, None]
}

grid_rf = GridSearchCV(
    rf,
    param_rf,
    cv=cv,
    scoring='accuracy',
    n_jobs=-1
)

grid_rf.fit(X_train_scaled, y_train)

print("RF Best Params:", grid_rf.best_params_)
print("RF CV Accuracy:", grid_rf.best_score_)


Best RF Parameters: {'max_depth': 10, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 200}
Best CV ROC-AUC: 0.8014112950415722


In [38]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000)

param_lr = {
    'C': [0.01, 0.1, 1, 10]
}

grid_lr = GridSearchCV(
    lr,
    param_lr,
    scoring='accuracy'
)

grid_lr.fit(X_train, y_train)

print("Logistic Best Params:", grid_lr.best_params_)
print("Logistic CV Accuracy:", grid_lr.best_score_)


Logistic Best Params: {'C': 0.01}
Logistic CV Accuracy: 0.7292193676262839


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)

param_gb = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5]
}

grid_gb = GridSearchCV(
    gb,
    param_gb,
    cv=cv,
    scoring='accuracy'
)

grid_gb.fit(X_train_scaled, y_train)

print("GB Best Params:", grid_gb.best_params_)
print("GB CV Accuracy:", grid_gb.best_score_)


Gradient Boosting Accuracy: 0.7257101238164603


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold

svm = SVC(probability=True)

param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['rbf', 'linear']
}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_svm = GridSearchCV(
    svm,
    param_grid_svm,
    cv=skf,
    scoring='roc_auc'
)

grid_svm.fit(X_train, y_train)

print("Best SVM Parameters:", grid_svm.best_params_)
print("Best CV ROC-AUC:", grid_svm.best_score_)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()

param_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

grid_knn = GridSearchCV(
    knn,
    param_knn,
    scoring='accuracy',
    cv=5
)

grid_knn.fit(X_train, y_train)

print("KNN Best Params:", grid_knn.best_params_)
print("KNN CV Accuracy:", grid_knn.best_score_)


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

nb = GaussianNB()

param_nb = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
}

grid_nb = GridSearchCV(
    nb,
    param_nb,
    scoring='accuracy',
    cv=5
)

grid_nb.fit(X_train, y_train)

print("Naive Bayes Best Params:", grid_nb.best_params_)
print("Naive Bayes CV Accuracy:", grid_nb.best_score_)


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, hgb.predict(X_test)))


              precision    recall  f1-score   support

           0       0.71      0.78      0.74      6937
           1       0.75      0.68      0.71      6793

    accuracy                           0.73     13730
   macro avg       0.73      0.73      0.73     13730
weighted avg       0.73      0.73      0.73     13730



In [ ]:
from sklearn.metrics import roc_auc_score
y_prob = rf.predict_proba(X_test)[:, 1]  

print("ROC-AUC:", roc_auc_score(y_test, y_prob))


ROC-AUC: 0.7964414796574779


In [79]:

df

,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio,cholesterol_2,cholesterol_3,gluc_2,gluc_3
0,-0.417920,1,0.444644,-0.845674,-0.997062,-0.138318,0,0,1,0,False,False,False,False
1,0.320682,0,-1.021516,0.759038,0.796150,0.919190,0,0,1,1,False,True,False,False
2,-0.270200,0,0.078104,-0.706134,0.198412,-1.195825,0,0,0,1,False,True,False,False
3,-0.713361,1,0.566823,0.549728,1.393887,1.976698,0,0,1,1,False,False,False,False
4,-0.861081,0,-1.021516,-1.264295,-1.594799,-2.253333,0,0,0,0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68644,-0.122480,1,0.444644,0.131107,-0.399325,-0.138318,1,0,1,0,False,False,False,False
68645,1.207004,0,-0.777156,3.619612,0.796150,0.919190,0,0,1,1,True,False,True,False
68646,-0.122480,1,2.277343,2.154440,3.187099,0.919190,0,1,0,1,False,True,False,False
68647,1.207004,0,-0.166256,-0.147973,0.497281,-0.138318,0,0,0,1,False,False,True,False


In [30]:
# After creating X_train, print column order
print("Training columns:", X_train.columns.tolist())
# This should match the order in app.py predict function

Training columns: ['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'smoke', 'alco', 'active', 'cholesterol_2', 'cholesterol_3', 'gluc_2', 'gluc_3']


In [31]:
df

,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio,cholesterol_2,cholesterol_3,gluc_2,gluc_3
0,50.0,1,168,62.0,110,80,0,0,1,0,False,False,False,False
1,55.0,0,156,85.0,140,90,0,0,1,1,False,True,False,False
2,51.0,0,165,64.0,130,70,0,0,0,1,False,True,False,False
3,48.0,1,169,82.0,150,100,0,0,1,1,False,False,False,False
4,47.0,0,156,56.0,100,60,0,0,0,0,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68644,52.0,1,168,76.0,120,80,1,0,1,0,False,False,False,False
68645,61.0,0,158,126.0,140,90,0,0,1,1,True,False,True,False
68646,52.0,1,183,105.0,180,90,0,1,0,1,False,True,False,False
68647,61.0,0,163,72.0,135,80,0,0,0,1,False,False,True,False
